In [1]:
import os
import pandas as pd
import re
import numpy as np
import datetime

In [2]:
def get_sku_sales_date(xls_file_name):
    date_pattern = r"\d{4}-\d{2}-\d{2}"
    sku_date = re.findall(date_pattern,xls_file_name)[0]
    return sku_date

In [3]:
bilibili_list = [730603253025,730901991727,730903103301]
def bilibili_taoke_divide(sku_id,bilibili_list,file_name):
    if sku_id in bilibili_list and ("定向计划" in file_name or "自主推广-商品维度-推广概览-数据分布" in file_name):
        return "B站"
    else:
        return "taoke"

In [4]:
itemList = ["塞德思","灵的融","希纳露","百热朗","健眠计划","棒维素","琵力消","整肠药"]
def tag_add(itemName,itemLists):
    for item in itemLists:
        if item in itemName:
            return item
        else:
            continue
    return itemName

In [5]:
xls_list=os.listdir("D:/盐野义/数据/taoke/")
df = pd.read_csv("D:/盐野义/数据/taoke/"+xls_list[0])
#df["渠道"] = df["商品ID"].apply(lambda x:bilibili_taoke_divide(sku_id = x,bilibili_list = bilibili_list,file_name = xls_list[0]))
df["日期"] = get_sku_sales_date(xls_file_name=xls_list[0])
df["文件名"] = xls_list[0]
for bilibili_csv in xls_list[1:]:
    bilibili_data_sub = pd.read_csv("D:/盐野义/数据/taoke/"+bilibili_csv)
    #bilibili_data_sub["渠道"] = bilibili_data_sub["商品ID"].apply(lambda x:bilibili_taoke_divide(sku_id= x,bilibili_list = bilibili_list,file_name = bilibili_csv))
    bilibili_data_sub["日期"] = get_sku_sales_date(xls_file_name=bilibili_csv)
    bilibili_data_sub["文件名"] = bilibili_csv
    df = pd.concat([df,bilibili_data_sub],axis=0,ignore_index=True)
df["商品名称"] = df["商品名称"].fillna('其他')
df["商品别名"] = df["商品名称"].apply(lambda x:tag_add(x,itemList))
df["渠道"] = df.apply(lambda row:bilibili_taoke_divide(sku_id=row["商品ID"],bilibili_list=bilibili_list,file_name=row["文件名"]),axis=1)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10248\1641066044.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,bilibili_data_sub],axis=0,ignore_index=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10248\1641066044.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,bilibili_data_sub],axis=0,ignore_index=True)


In [6]:
bilibili_data = df[df["渠道"]=="B站"]
taoke_data = df[df["渠道"]=="taoke"]

In [7]:
bilibili_pivot = bilibili_data.pivot_table(values="付款人数",index="商品别名",columns="日期",aggfunc="sum")
bilibili_sort_dict ={'希纳露':1,"灵的融":0}
bilibili_pivot.sort_index(key=lambda x:x.map(bilibili_sort_dict),inplace=True)
taoke_pivot = taoke_data.pivot_table(values="付款人数",index="商品别名",columns="日期",aggfunc="sum")
taoke_sort_dict ={"塞德思":0,"灵的融":1,"百热朗":2,"希纳露":3,"棒维素":4,"琵力消":5,"健眠计划":6}
taoke_pivot.sort_index(key=lambda x:x.map(taoke_sort_dict),inplace=True)


In [8]:
bilibili_pivot.to_excel("D:/盐野义/数据/每周淘客/bilibili推广_"+str(datetime.date.today())+".xlsx")
taoke_pivot.to_excel("D:/盐野义/数据/每周淘客/taoke推广_"+str(datetime.date.today())+".xlsx")